#  Functional annotations by the Enformer model

Copyright 2021 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

This colab showcases the usage of the Enformer model published in

**"Effective gene expression prediction from sequence by integrating long-range interactions"**

Žiga Avsec, Vikram Agarwal, Daniel Visentin, Joseph R. Ledsam, Agnieszka Grabska-Barwinska, Kyle R. Taylor, Yannis Assael, John Jumper, Pushmeet Kohli, David R. Kelley

**Note:** This colab will not yet work since the model isn't yet publicly available. We are working on enabling this and will update the colab accordingly.

## Setup

**Start the colab kernel with GPU**: Runtime -> Change runtime type -> GPU

In [ ]:
import tensorflow as tf
# Make sure the GPU is enabled
assert tf.config.list_physical_devices('GPU'), 'Start the colab kernel with GPU: Runtime -> Change runtime type -> GPU'

In [ ]:
!pip install kipoiseq==0.5.2 --quiet > /dev/null
# You can ignore the pyYAML error

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jsonschema 4.23.0 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.
referencing 0.35.1 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.


### Imports

In [ ]:
import tensorflow_hub as hub
import joblib
import gzip
import kipoiseq
from kipoiseq import Interval
import pyfaidx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
transform_path = 'gs://dm-enformer/models/enformer.finetuned.SAD.robustscaler-PCA500-robustscaler.transform.pkl'
model_path = 'https://tfhub.dev/deepmind/enformer/1'
fasta_file = '/root/data/genome.fa'
clinvar_vcf = '/root/data/clinvar.vcf.gz'

In [ ]:
# Download targets from Basenji2 dataset
# Cite: Kelley et al Cross-species regulatory sequence activity prediction. PLoS Comput. Biol. 16, e1008050 (2020).
targets_txt = 'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_human.txt'
df_targets = pd.read_csv(targets_txt, sep='\t')
df_targets.head(3)

,index,genome,identifier,file,clip,scale,sum_stat,description
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion


### Download files

Download and index the reference genome fasta file

Credit to Genome Reference Consortium: https://www.ncbi.nlm.nih.gov/grc

Schneider et al 2017 http://dx.doi.org/10.1101/gr.213611.116: Evaluation of GRCh38 and de novo haploid genome assemblies demonstrates the enduring quality of the reference assembly

In [ ]:
!mkdir -p /root/data
!wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > {fasta_file}
pyfaidx.Faidx(fasta_file)
!ls /root/data

--2024-10-28 13:41:57--  http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 938.09M  21.9MB/s    in 47s     

2024-10-28 13:42:44 (19.9 MB/s) - written to stdout [983659424/983659424]

genome.fa  genome.fa.fai


Download the clinvar file. Reference:

Landrum MJ, Lee JM, Benson M, Brown GR, Chao C, Chitipiralla S, Gu B, Hart J, Hoffman D, Jang W, Karapetyan K, Katz K, Liu C, Maddipatla Z, Malheiro A, McDaniel K, Ovetsky M, Riley G, Zhou G, Holmes JB, Kattman BL, Maglott DR. ClinVar: improving access to variant interpretations and supporting evidence. Nucleic Acids Res . 2018 Jan 4. PubMed PMID: 29165669 .


In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz -O /root/data/clinvar.vcf.gz

--2024-10-28 13:43:36--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.31, 130.14.250.7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104431734 (100M) [application/x-gzip]
Saving to: ‘/root/data/clinvar.vcf.gz’

/root/data/clinvar. 100%[===================>]  99.59M  33.8MB/s    in 2.9s    

2024-10-28 13:43:39 (33.8 MB/s) - ‘/root/data/clinvar.vcf.gz’ saved [104431734/104431734]



### Code (double click on the title to show the code)

In [ ]:
# @title `Enformer`, `EnformerScoreVariantsNormalized`, `EnformerScoreVariantsPCANormalized`,
SEQUENCE_LENGTH = 393216

class Enformer:

  def __init__(self, tfhub_url):
    self._model = hub.load(tfhub_url).model

  def predict_on_batch(self, inputs):
    predictions = self._model.predict_on_batch(inputs)
    return {k: v.numpy() for k, v in predictions.items()}

  @tf.function
  def contribution_input_grad(self, input_sequence,
                              target_mask, output_head='human'):
    input_sequence = input_sequence[tf.newaxis]

    target_mask_mass = tf.reduce_sum(target_mask)
    with tf.GradientTape() as tape:
      tape.watch(input_sequence)
      prediction = tf.reduce_sum(
          target_mask[tf.newaxis] *
          self._model.predict_on_batch(input_sequence)[output_head]) / target_mask_mass

    input_grad = tape.gradient(prediction, input_sequence) * input_sequence
    input_grad = tf.squeeze(input_grad, axis=0)
    return tf.reduce_sum(input_grad, axis=-1)


class EnformerScoreVariantsRaw:

  def __init__(self, tfhub_url, organism='human'):
    self._model = Enformer(tfhub_url)
    self._organism = organism

  def predict_on_batch(self, inputs):
    ref_prediction = self._model.predict_on_batch(inputs['ref'])[self._organism]
    alt_prediction = self._model.predict_on_batch(inputs['alt'])[self._organism]

    return alt_prediction.mean(axis=1) - ref_prediction.mean(axis=1)


class EnformerScoreVariantsNormalized:

  def __init__(self, tfhub_url, transform_pkl_path,
               organism='human'):
    assert organism == 'human', 'Transforms only compatible with organism=human'
    self._model = EnformerScoreVariantsRaw(tfhub_url, organism)
    with tf.io.gfile.GFile(transform_pkl_path, 'rb') as f:
      transform_pipeline = joblib.load(f)
    self._transform = transform_pipeline.steps[0][1]  # StandardScaler.

  def predict_on_batch(self, inputs):
    scores = self._model.predict_on_batch(inputs)
    return self._transform.transform(scores)


class EnformerScoreVariantsPCANormalized:

  def __init__(self, tfhub_url, transform_pkl_path,
               organism='human', num_top_features=500):
    self._model = EnformerScoreVariantsRaw(tfhub_url, organism)
    with tf.io.gfile.GFile(transform_pkl_path, 'rb') as f:
      self._transform = joblib.load(f)
    self._num_top_features = num_top_features

  def predict_on_batch(self, inputs):
    scores = self._model.predict_on_batch(inputs)
    return self._transform.transform(scores)[:, :self._num_top_features]


# TODO(avsec): Add feature description: Either PCX, or full names.

In [ ]:
# @title `variant_centered_sequences`

class FastaStringExtractor:

    def __init__(self, fasta_file):
        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval: Interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                          trimmed_interval.start + 1,
                                          trimmed_interval.stop).seq).upper()
        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()


def variant_generator(vcf_file, gzipped=False):
  """Yields a kipoiseq.dataclasses.Variant for each row in VCF file."""
  def _open(file):
    return gzip.open(vcf_file, 'rt') if gzipped else open(vcf_file)

  with _open(vcf_file) as f:
    for line in f:
      if line.startswith('#'):
        continue
      chrom, pos, id, ref, alt_list = line.split('\t')[:5]
      # Split ALT alleles and return individual variants as output.
      for alt in alt_list.split(','):
        yield kipoiseq.dataclasses.Variant(chrom=chrom, pos=pos,
                                           ref=ref, alt=alt, id=id)


def one_hot_encode(sequence):
  return kipoiseq.transforms.functional.one_hot_dna(sequence).astype(np.float32)


def variant_centered_sequences(vcf_file, sequence_length, gzipped=False,
                               chr_prefix=''):
  seq_extractor = kipoiseq.extractors.VariantSeqExtractor(
    reference_sequence=FastaStringExtractor(fasta_file))

  for variant in variant_generator(vcf_file, gzipped=gzipped):
    interval = Interval(chr_prefix + variant.chrom,
                        variant.pos, variant.pos)
    interval = interval.resize(sequence_length)
    center = interval.center() - interval.start

    reference = seq_extractor.extract(interval, [], anchor=center)
    alternate = seq_extractor.extract(interval, [variant], anchor=center)

    yield {'inputs': {'ref': one_hot_encode(reference),
                      'alt': one_hot_encode(alternate)},
           'metadata': {'chrom': chr_prefix + variant.chrom,
                        'pos': variant.pos,
                        'id': variant.id,
                        'ref': variant.ref,
                        'alt': variant.alt}}

In [ ]:
# @title `plot_tracks`

def plot_tracks(tracks, interval, height=1.5):
  fig, axes = plt.subplots(len(tracks), 1, figsize=(20, height * len(tracks)), sharex=True)
  for ax, (title, y) in zip(axes, tracks.items()):
    ax.fill_between(np.linspace(interval.start, interval.end, num=len(y)), y)
    ax.set_title(title)
    sns.despine(top=True, right=True, bottom=True)
  ax.set_xlabel(str(interval))
  plt.tight_layout()

## Make predictions for a genetic sequenece

In [ ]:
model = Enformer(model_path)

fasta_extractor = FastaStringExtractor(fasta_file)

## Score variants in a VCF file

### Report top 100 PCs

In [ ]:
enformer_score_variants = EnformerScoreVariantsPCANormalized(model_path, transform_path, num_top_features=100)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 0.23.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TruncatedSVD from version 0.23.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 1.5.2. This might lead to breaking code or invalid

In [ ]:
# Score the first 5 variants from ClinVar
# Lower-dimensional scores (100 PCs)
it = variant_centered_sequences(clinvar_vcf, sequence_length=SEQUENCE_LENGTH,
                                gzipped=True, chr_prefix='chr')
example_list = []
for i, example in enumerate(it):
  if i >= 5:
    break
  variant_scores = enformer_score_variants.predict_on_batch(
      {k: v[tf.newaxis] for k,v in example['inputs'].items()})[0]
  variant_scores = {f'PC{i}': score for i, score in enumerate(variant_scores)}
  example_list.append({**example['metadata'],
                       **variant_scores})
  if i % 2 == 0:
    print(f'Done {i}')
df_enf = pd.DataFrame(example_list)
df_enf

Done 0
Done 2
Done 4


,chrom,pos,id,ref,alt,PC0,PC1,PC2,PC3,PC4,...,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99
0,chr1,69134,2205837,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,-1.328947,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206
1,chr1,69314,3205580,T,G,-4.572755,7.086992,-8.081913,10.567889,2.176969,...,-2.290082,-1.098749,-3.132009,-2.266651,-1.732427,3.386143,-3.429577,3.390267,1.662880,-1.753280
2,chr1,69423,3205581,G,A,-2.696827,-1.338069,0.444169,0.210701,-0.563864,...,0.239099,0.458585,-0.027790,0.700127,0.093348,-0.014985,0.401269,-0.407513,0.363768,0.553934
3,chr1,69581,2252161,C,G,-2.235523,4.260818,-6.350739,2.498391,-2.610248,...,-4.241679,-0.514959,1.946530,-2.963845,-3.155007,-0.040376,-0.319441,1.259213,-2.399494,-1.180045
4,chr1,69682,2396347,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.364287,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958


# Calculate Distance to Tss for variant-gene pairs

## Setup

In [ ]:
%%time
import pandas as pd
import os
import glob
import numpy as np
from typing import Dict, Text
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm
import tensorflow as tf
import keras
from keras.models import load_model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
import keras.backend as K

CPU times: user 55 µs, sys: 6 µs, total: 61 µs
Wall time: 66 µs


In [ ]:
!wget https://storage.googleapis.com/adult-gtex/references/v8/reference-tables/gencode.v26.GRCh38.genes.gtf | -O /root/data/gencode.v26.GRCh38.genes.gtf

/bin/bash: line 1: -O: command not found
--2024-10-28 13:46:51--  https://storage.googleapis.com/adult-gtex/references/v8/reference-tables/gencode.v26.GRCh38.genes.gtf
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134502408 (128M) [application/octet-stream]
Saving to: ‘gencode.v26.GRCh38.genes.gtf.2’

gencode.v26.GRCh38. 100%[===================>] 128.27M  64.3MB/s    in 2.0s    

2024-10-28 13:46:53 (64.3 MB/s) - ‘gencode.v26.GRCh38.genes.gtf.2’ saved [134502408/134502408]



In [ ]:
#check the gtf file:
!head -n 7 ./gencode.v26.GRCh38.genes.gtf

##description: evidence-based annotation of the human genome (GRCh38), version 26 (Ensembl 88)
##provider: GENCODE
##contact: gencode-help@sanger.ac.uk
##format: gtf
##date: 2017-03-14
##collapsed version generated by GTEx LDACC
chr1	HAVANA	gene	11869	14403	.	+	.	gene_id "ENSG00000223972.5"; transcript_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; transcript_type "transcribed_unprocessed_pseudogene"; transcript_name "DDX11L1"; level 2; havana_gene "OTTHUMG00000000961.2";


In [ ]:
hdr = ["chr","source","type","gene_left","gene_right", ".", "strand","..", "metadata"] #just the header to read gtf
gtf = pd.read_csv("./gencode.v26.GRCh38.genes.gtf", sep='\t', skiprows=6, header=None)#gtf downloaded from GTEx portal
gtf.columns = hdr
gtf = gtf[gtf.type=="gene"] #50K genes
gtf.head() #check

,chr,source,type,gene_left,gene_right,.,strand,..,metadata
0,chr1,HAVANA,gene,11869,14403,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
6,chr1,HAVANA,gene,14410,29553,.,-,.,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
19,chr1,ENSEMBL,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267.1""; transcript_id ""EN..."
22,chr1,HAVANA,gene,29571,31109,.,+,.,"gene_id ""ENSG00000243485.5""; transcript_id ""EN..."
28,chr1,HAVANA,gene,34554,36081,.,-,.,"gene_id ""ENSG00000237613.2""; transcript_id ""EN..."


In [ ]:
#check this metadata column:
gtf.metadata.values[0]

'gene_id "ENSG00000223972.5"; transcript_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; transcript_type "transcribed_unprocessed_pseudogene"; transcript_name "DDX11L1"; level 2; havana_gene "OTTHUMG00000000961.2";'

In [ ]:
#annotate tss, which is the left for plus strand and right for minus strand
gtf["tss_position"] = -1 #dummy
gtf.loc[gtf.strand=="-", "tss_position"] = gtf.gene_right
gtf.loc[gtf.strand=="+", "tss_position"] = gtf.gene_left
#annotate ENSG ID
gtf["ensg_id"] = gtf.metadata.str.split(";").str[0].str.split('"').str[1]
gtf["gene_name"] = gtf.metadata.str.split(";").str[3].str.split('"').str[1]

#write the file:
gtf[["chr", "tss_position", "ensg_id", "gene_name"]].to_csv("./gencode.v26.GRCh38.genes.tssposition.tsv", sep='\t', index=False)

In [ ]:
#checking the tss position file:

#function to get ENSG ID list:
df = pd.read_csv("./gencode.v26.GRCh38.genes.tssposition.tsv", sep='\t')
df

,chr,tss_position,ensg_id,gene_name
0,chr1,11869,ENSG00000223972.5,DDX11L1
1,chr1,29553,ENSG00000227232.5,WASH7P
2,chr1,17436,ENSG00000278267.1,MIR6859-1
3,chr1,29571,ENSG00000243485.5,MIR1302-2HG
4,chr1,36081,ENSG00000237613.2,FAM138A
...,...,...,...,...
56195,chrM,14673,ENSG00000198695.2,MT-ND6
56196,chrM,14742,ENSG00000210194.1,MT-TE
56197,chrM,14747,ENSG00000198727.2,MT-CYB
56198,chrM,15888,ENSG00000210195.2,MT-TT


In [ ]:
#split per chr to make it faster when annotating.
tss_dict = {}
for chr in df.chr.unique():
    tss_dict[chr] = df[df.chr==chr]
tss_dict

{'chr1':        chr  tss_position             ensg_id    gene_name
 0     chr1         11869   ENSG00000223972.5      DDX11L1
 1     chr1         29553   ENSG00000227232.5       WASH7P
 2     chr1         17436   ENSG00000278267.1    MIR6859-1
 3     chr1         29571   ENSG00000243485.5  MIR1302-2HG
 4     chr1         36081   ENSG00000237613.2      FAM138A
 ...    ...           ...                 ...          ...
 5051  chr1     248859085  ENSG00000171163.15       ZNF692
 5052  chr1     248859164   ENSG00000227237.1   AL672294.1
 5053  chr1     248906196  ENSG00000185220.11        PGBD2
 5054  chr1     248912795   ENSG00000200495.1   RNU6-1205P
 5055  chr1     248936581   ENSG00000233084.2    RPL23AP25
 
 [5056 rows x 4 columns],
 'chr2':        chr  tss_position             ensg_id   gene_name
 5056  chr2         46870   ENSG00000184731.5     FAM110C
 5057  chr2        197569   ENSG00000227061.1  AC079779.7
 5058  chr2        266398  ENSG00000035115.21      SH3YL1
 5059  chr2     

In [ ]:
df_enf['id'] = df_enf['chrom'] + '_' + df_enf['pos'].astype(str) + '_' + df_enf['ref'] + '_' + df_enf['alt']
df_enf

,chrom,pos,id,ref,alt,PC0,PC1,PC2,PC3,PC4,...,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99
0,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,-1.328947,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206
1,chr1,69314,chr1_69314_T_G,T,G,-4.572755,7.086992,-8.081913,10.567889,2.176969,...,-2.290082,-1.098749,-3.132009,-2.266651,-1.732427,3.386143,-3.429577,3.390267,1.662880,-1.753280
2,chr1,69423,chr1_69423_G_A,G,A,-2.696827,-1.338069,0.444169,0.210701,-0.563864,...,0.239099,0.458585,-0.027790,0.700127,0.093348,-0.014985,0.401269,-0.407513,0.363768,0.553934
3,chr1,69581,chr1_69581_C_G,C,G,-2.235523,4.260818,-6.350739,2.498391,-2.610248,...,-4.241679,-0.514959,1.946530,-2.963845,-3.155007,-0.040376,-0.319441,1.259213,-2.399494,-1.180045
4,chr1,69682,chr1_69682_G_A,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.364287,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958


In [ ]:
df_vcf = df_enf.iloc[:,:5]
df_vcf.columns=['chr', 'pos','id','ref','alt']
df_vcf

,chr,pos,id,ref,alt
0,chr1,69134,chr1_69134_A_G,A,G
1,chr1,69314,chr1_69314_T_G,T,G
2,chr1,69423,chr1_69423_G_A,G,A
3,chr1,69581,chr1_69581_C_G,C,G
4,chr1,69682,chr1_69682_G_A,G,A


In [ ]:
%%time
#definee the functino that
def return_vg(chr, pos): #input: chr and pos, output: dataframe of chr, pos, tss_distance, ensg_id, gene_name
    st = tss_dict[chr]
    st["tss_distance"] = pos - st.tss_position #negative = the variant upstream in the genome 逆かも？need to check
    st = st[abs(st.tss_distance)<10**6]
    st["pos"] = pos
    cols = ["chr","pos","tss_distance","ensg_id","gene_name"]
    return (st[cols])
def convert_to_vg(vcf): #input: vcf-like tsv, output: expanded tsv where row = variant-gene pair
    out = []
    for i in range(vcf.shape[0]):
        tmp = return_vg(vcf.chr[i], vcf.pos[i])
        tmp[vcf.columns] = vcf.iloc[i,:]
        out.append(tmp)
    return (pd.concat(out))

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 5.96 µs


In [ ]:
%%time
#check that the output is good:
df_tss = convert_to_vg(df_vcf)
df_tss["ensg_id"] = df_tss["ensg_id"].str.split('.', expand=True)[0]
df_tss

CPU times: user 33.3 ms, sys: 0 ns, total: 33.3 ms
Wall time: 38.5 ms


<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
<timed exec>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be t

,chr,pos,tss_distance,ensg_id,gene_name,id,ref,alt
0,chr1,69134,57265,ENSG00000223972,DDX11L1,chr1_69134_A_G,A,G
1,chr1,69134,39581,ENSG00000227232,WASH7P,chr1_69134_A_G,A,G
2,chr1,69134,51698,ENSG00000278267,MIR6859-1,chr1_69134_A_G,A,G
3,chr1,69134,39563,ENSG00000243485,MIR1302-2HG,chr1_69134_A_G,A,G
4,chr1,69134,33053,ENSG00000237613,FAM138A,chr1_69134_A_G,A,G
...,...,...,...,...,...,...,...,...
65,chr1,69682,-938511,ENSG00000231702,RP11-54O7.10,chr1_69682_G_A,G,A
66,chr1,69682,-943511,ENSG00000224969,RP11-54O7.11,chr1_69682_G_A,G,A
67,chr1,69682,-950441,ENSG00000188157,AGRN,chr1_69682_G_A,G,A
68,chr1,69682,-990052,ENSG00000217801,RP11-465B22.3,chr1_69682_G_A,G,A


# EMS for GTEx 49 tissues

##Data Preprocessing

In [ ]:
df_enf

,chrom,pos,id,ref,alt,PC0,PC1,PC2,PC3,PC4,...,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99
0,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,-1.328947,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206
1,chr1,69314,chr1_69314_T_G,T,G,-4.572755,7.086992,-8.081913,10.567889,2.176969,...,-2.290082,-1.098749,-3.132009,-2.266651,-1.732427,3.386143,-3.429577,3.390267,1.662880,-1.753280
2,chr1,69423,chr1_69423_G_A,G,A,-2.696827,-1.338069,0.444169,0.210701,-0.563864,...,0.239099,0.458585,-0.027790,0.700127,0.093348,-0.014985,0.401269,-0.407513,0.363768,0.553934
3,chr1,69581,chr1_69581_C_G,C,G,-2.235523,4.260818,-6.350739,2.498391,-2.610248,...,-4.241679,-0.514959,1.946530,-2.963845,-3.155007,-0.040376,-0.319441,1.259213,-2.399494,-1.180045
4,chr1,69682,chr1_69682_G_A,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.364287,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958


In [ ]:
df_tss.drop(df_tss.columns.difference(['id', 'ensg_id', 'tss_distance']), axis=1, inplace=True)
df_tss

,tss_distance,ensg_id,id
0,57265,ENSG00000223972,chr1_69134_A_G
1,39581,ENSG00000227232,chr1_69134_A_G
2,51698,ENSG00000278267,chr1_69134_A_G
3,39563,ENSG00000243485,chr1_69134_A_G
4,33053,ENSG00000237613,chr1_69134_A_G
...,...,...,...
65,-938511,ENSG00000231702,chr1_69682_G_A
66,-943511,ENSG00000224969,chr1_69682_G_A
67,-950441,ENSG00000188157,chr1_69682_G_A
68,-990052,ENSG00000217801,chr1_69682_G_A


In [ ]:
X_data = pd.merge(df_enf, df_tss, how="left",on="id")
X_data

,chrom,pos,id,ref,alt,PC0,PC1,PC2,PC3,PC4,...,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,tss_distance,ensg_id
0,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,57265,ENSG00000223972
1,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,39581,ENSG00000227232
2,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,51698,ENSG00000278267
3,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,39563,ENSG00000243485
4,chr1,69134,chr1_69134_A_G,A,G,-7.434000,2.266127,-4.216179,7.759473,1.583225,...,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,33053,ENSG00000237613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,chr1,69682,chr1_69682_G_A,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-938511,ENSG00000231702
346,chr1,69682,chr1_69682_G_A,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-943511,ENSG00000224969
347,chr1,69682,chr1_69682_G_A,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-950441,ENSG00000188157
348,chr1,69682,chr1_69682_G_A,G,A,-8.017408,4.184911,-6.303721,6.835107,-0.562544,...,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-990052,ENSG00000217801


In [ ]:
col = ['id', 'ensg_id'] + [f'PC{i}' for i in range(100)] + ['tss_distance']
X_data = X_data[col]
X_data

,id,ensg_id,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,...,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,tss_distance
0,chr1_69134_A_G,ENSG00000223972,-7.434000,2.266127,-4.216179,7.759473,1.583225,0.776162,-1.078389,-5.642815,...,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,57265
1,chr1_69134_A_G,ENSG00000227232,-7.434000,2.266127,-4.216179,7.759473,1.583225,0.776162,-1.078389,-5.642815,...,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,39581
2,chr1_69134_A_G,ENSG00000278267,-7.434000,2.266127,-4.216179,7.759473,1.583225,0.776162,-1.078389,-5.642815,...,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,51698
3,chr1_69134_A_G,ENSG00000243485,-7.434000,2.266127,-4.216179,7.759473,1.583225,0.776162,-1.078389,-5.642815,...,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,39563
4,chr1_69134_A_G,ENSG00000237613,-7.434000,2.266127,-4.216179,7.759473,1.583225,0.776162,-1.078389,-5.642815,...,-1.112461,0.720135,-2.699648,-1.532841,0.802330,-1.897562,1.226114,1.119790,0.277206,33053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,chr1_69682_G_A,ENSG00000231702,-8.017408,4.184911,-6.303721,6.835107,-0.562544,-0.167148,-4.245413,-5.228899,...,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-938511
346,chr1_69682_G_A,ENSG00000224969,-8.017408,4.184911,-6.303721,6.835107,-0.562544,-0.167148,-4.245413,-5.228899,...,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-943511
347,chr1_69682_G_A,ENSG00000188157,-8.017408,4.184911,-6.303721,6.835107,-0.562544,-0.167148,-4.245413,-5.228899,...,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-950441
348,chr1_69682_G_A,ENSG00000217801,-8.017408,4.184911,-6.303721,6.835107,-0.562544,-0.167148,-4.245413,-5.228899,...,-0.108076,-1.133553,-1.795635,-2.662625,0.986908,-2.897970,0.544191,0.908301,-0.798958,-990052


In [ ]:
sign = np.sign(X_data.iloc[:, 2:102].values)
X_data.iloc[:,2:] = X_data.iloc[:,2:].apply(lambda x: np.log(abs(x)+1))
X_data.iloc[:,2:102] = X_data.iloc[:,2:102] * sign
X_data

<ipython-input-56-fbed39ce0d85>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      10.955462
1      10.586130
2      10.853194
3      10.585675
4      10.405898
         ...    
345    13.752051
346    13.757364
347    13.764682
348    13.805514
349    13.809097
Name: tss_distance, Length: 350, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_data.iloc[:,2:] = X_data.iloc[:,2:].apply(lambda x: np.log(abs(x)+1))


,id,ensg_id,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,...,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,tss_distance
0,chr1_69134_A_G,ENSG00000223972,-2.132271,1.183605,-1.651765,2.170136,0.949039,0.574455,-0.731593,-1.893536,...,-0.747853,0.542403,-1.308238,-0.929341,0.589081,-1.063870,0.800258,0.751317,0.244675,10.955462
1,chr1_69134_A_G,ENSG00000227232,-2.132271,1.183605,-1.651765,2.170136,0.949039,0.574455,-0.731593,-1.893536,...,-0.747853,0.542403,-1.308238,-0.929341,0.589081,-1.063870,0.800258,0.751317,0.244675,10.586130
2,chr1_69134_A_G,ENSG00000278267,-2.132271,1.183605,-1.651765,2.170136,0.949039,0.574455,-0.731593,-1.893536,...,-0.747853,0.542403,-1.308238,-0.929341,0.589081,-1.063870,0.800258,0.751317,0.244675,10.853194
3,chr1_69134_A_G,ENSG00000243485,-2.132271,1.183605,-1.651765,2.170136,0.949039,0.574455,-0.731593,-1.893536,...,-0.747853,0.542403,-1.308238,-0.929341,0.589081,-1.063870,0.800258,0.751317,0.244675,10.585675
4,chr1_69134_A_G,ENSG00000237613,-2.132271,1.183605,-1.651765,2.170136,0.949039,0.574455,-0.731593,-1.893536,...,-0.747853,0.542403,-1.308238,-0.929341,0.589081,-1.063870,0.800258,0.751317,0.244675,10.405898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,chr1_69682_G_A,ENSG00000231702,-2.199157,1.645753,-1.988384,2.058614,-0.446315,-0.154563,-1.657354,-1.829200,...,-0.102625,-0.757789,-1.028059,-1.298180,0.686579,-1.360456,0.434500,0.646213,-0.587208,13.752051
346,chr1_69682_G_A,ENSG00000224969,-2.199157,1.645753,-1.988384,2.058614,-0.446315,-0.154563,-1.657354,-1.829200,...,-0.102625,-0.757789,-1.028059,-1.298180,0.686579,-1.360456,0.434500,0.646213,-0.587208,13.757364
347,chr1_69682_G_A,ENSG00000188157,-2.199157,1.645753,-1.988384,2.058614,-0.446315,-0.154563,-1.657354,-1.829200,...,-0.102625,-0.757789,-1.028059,-1.298180,0.686579,-1.360456,0.434500,0.646213,-0.587208,13.764682
348,chr1_69682_G_A,ENSG00000217801,-2.199157,1.645753,-1.988384,2.058614,-0.446315,-0.154563,-1.657354,-1.829200,...,-0.102625,-0.757789,-1.028059,-1.298180,0.686579,-1.360456,0.434500,0.646213,-0.587208,13.805514


## Prediction Scores

In [ ]:
# Custom loss function for EMS model training and prediction
def custom_canonical_mse(y_true_pip, y_pred, mask_val=-2): #input y_true_pip is now a vector of length N and is pip, for pos and neg and is masked otherwise
    mask = K.cast(K.not_equal(y_true_pip, mask_val), K.floatx()) #this is the mask
    #first, create binary label:
    y_bin = tf.where(tf.greater(y_true_pip, thres),1.0, y_true_pip)
    y_bin = tf.where(tf.less(y_bin, 0.0001),0.0, y_bin) #the mask -2 will be gone, but this should be fine if we mask first.
    #and the weight
    sumpos = tf.math.reduce_sum(tf.where(tf.greater(y_true_pip, thres),y_true_pip, 0), axis=0) #this gives the sum(PIP) for pos.
    nneg = tf.math.reduce_sum(tf.cast(tf.less(y_true_pip, 0.0001), tf.float32)*mask, axis=0) #this gives the num. neg
    neg_weight = sumpos/nneg
    weight = tf.where(tf.greater(y_true_pip, thres), y_true_pip, neg_weight) #positive weight = PIP itself, negative weight = sum(PIP in pos)/nneg
    loss = K.square(y_pred - y_bin) #loss of the binary prediction
    loss = K.sum(K.sum(loss*weight*mask, axis=1)) #loss is a scaler. sum over all tissues
    return loss

In [ ]:
# Load the pretrained EMSv2 model
!wget https://github.com/ytakahashi-statgen/expression_modifier_score_v2/raw/refs/heads/main/ems_model.h5 | -O /root/data/ems_model.h5
model = load_model('./ems_model.h5', custom_objects={"custom_canonical_mse":custom_canonical_mse})

tissues = ['Whole_Blood', 'Muscle_Skeletal', 'Liver', 'Brain_Cerebellum','Prostate', 'Spleen', 'Skin_Sun_Exposed_Lower_leg', 'Artery_Coronary',
       'Esophagus_Muscularis', 'Esophagus_Gastroesophageal_Junction','Artery_Tibial', 'Heart_Atrial_Appendage', 'Nerve_Tibial',
       'Heart_Left_Ventricle', 'Adrenal_Gland', 'Adipose_Visceral_Omentum','Pancreas', 'Lung', 'Pituitary',
       'Brain_Nucleus_accumbens_basal_ganglia', 'Colon_Transverse','Adipose_Subcutaneous', 'Esophagus_Mucosa', 'Brain_Cortex', 'Thyroid',
       'Stomach', 'Breast_Mammary_Tissue', 'Colon_Sigmoid','Skin_Not_Sun_Exposed_Suprapubic', 'Testis', 'Artery_Aorta',
       'Brain_Amygdala', 'Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia', 'Brain_Cerebellar_Hemisphere',
       'Brain_Frontal_Cortex_BA9', 'Brain_Hippocampus', 'Brain_Hypothalamus','Brain_Putamen_basal_ganglia', 'Brain_Spinal_cord_cervical_c-1',
       'Brain_Substantia_nigra', 'Cells_Cultured_fibroblasts','Cells_EBV-transformed_lymphocytes', 'Kidney_Cortex',
       'Minor_Salivary_Gland', 'Ovary', 'Small_Intestine_Terminal_Ileum','Uterus', 'Vagina']

/bin/bash: line 1: -O: command not found
--2024-10-28 13:48:24--  https://github.com/ytakahashi-statgen/expression_modifier_score_v2/raw/refs/heads/main/ems_model.h5
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ytakahashi-statgen/expression_modifier_score_v2/refs/heads/main/ems_model.h5 [following]
--2024-10-28 13:48:25--  https://raw.githubusercontent.com/ytakahashi-statgen/expression_modifier_score_v2/refs/heads/main/ems_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4587704 (4.4M) [application/octet-stream]
Saving to: ‘ems_model.h5.5’

ems_model.h5.5      100%[===================

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
# Perform prediction and format as DataFrame
y_prob = model.predict(X_data.iloc[:, 2:])
y_prob = pd.DataFrame(y_prob,columns=tissues,index=X_data.index)
y_prob = pd.concat([X_data.iloc[:, :2], y_prob], axis=1)
y_prob

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


,id,ensg_id,Whole_Blood,Muscle_Skeletal,Liver,Brain_Cerebellum,Prostate,Spleen,Skin_Sun_Exposed_Lower_leg,Artery_Coronary,...,Brain_Spinal_cord_cervical_c-1,Brain_Substantia_nigra,Cells_Cultured_fibroblasts,Cells_EBV-transformed_lymphocytes,Kidney_Cortex,Minor_Salivary_Gland,Ovary,Small_Intestine_Terminal_Ileum,Uterus,Vagina
0,chr1_69134_A_G,ENSG00000223972,0.758650,0.797900,0.865915,0.850588,0.899836,0.852933,0.832310,0.892038,...,0.894990,0.892984,0.799175,0.840384,0.830139,0.890694,0.889961,0.886393,0.919453,0.914338
1,chr1_69134_A_G,ENSG00000227232,0.762210,0.803708,0.868973,0.854752,0.903980,0.857267,0.836524,0.895316,...,0.898649,0.897268,0.803904,0.841977,0.833226,0.894447,0.893466,0.889632,0.922235,0.916546
2,chr1_69134_A_G,ENSG00000278267,0.760042,0.800018,0.867007,0.852057,0.901317,0.854544,0.833868,0.893274,...,0.896383,0.894422,0.800936,0.841129,0.831164,0.891970,0.891150,0.887519,0.920374,0.915040
3,chr1_69134_A_G,ENSG00000243485,0.762212,0.803712,0.868976,0.854756,0.903983,0.857271,0.836527,0.895319,...,0.898651,0.897272,0.803907,0.841977,0.833229,0.894451,0.893469,0.889635,0.922237,0.916549
4,chr1_69134_A_G,ENSG00000237613,0.762922,0.805243,0.869772,0.855923,0.905096,0.858307,0.837555,0.896040,...,0.899412,0.898613,0.805043,0.841958,0.834179,0.895573,0.894511,0.890538,0.923100,0.917294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,chr1_69682_G_A,ENSG00000231702,0.284123,0.204791,0.420081,0.401502,0.302060,0.257562,0.255200,0.220010,...,0.285293,0.345012,0.233219,0.392412,0.535369,0.314204,0.458757,0.320268,0.370927,0.501528
346,chr1_69682_G_A,ENSG00000224969,0.282463,0.203531,0.418875,0.400182,0.300472,0.256378,0.253763,0.218607,...,0.283889,0.343504,0.232006,0.391113,0.534821,0.312338,0.457302,0.318739,0.369340,0.499488
347,chr1_69682_G_A,ENSG00000188157,0.280172,0.201797,0.417206,0.398357,0.298279,0.254747,0.251785,0.216675,...,0.281954,0.341422,0.230334,0.389317,0.534059,0.309764,0.455286,0.316630,0.367142,0.496652
348,chr1_69682_G_A,ENSG00000217801,0.267366,0.192177,0.407747,0.388029,0.285948,0.245619,0.240777,0.205912,...,0.271119,0.329723,0.220993,0.379153,0.529667,0.295357,0.443774,0.304802,0.354655,0.480316


In [ ]:
!wget https://github.com/ytakahashi-statgen/expression_modifier_score_v2/raw/refs/heads/main/ems_scaling_scores.csv | -O /root/data/ems_scaling_scores.csv

df_ems_scale = pd.read_csv(f"./ems_scaling_scores.csv", delimiter=",",index_col=0)
df_ems_scale

/bin/bash: line 1: -O: command not found
--2024-10-28 13:51:00--  https://github.com/ytakahashi-statgen/expression_modifier_score_v2/raw/refs/heads/main/ems_scaling_scores.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ytakahashi-statgen/expression_modifier_score_v2/refs/heads/main/ems_scaling_scores.csv [following]
--2024-10-28 13:51:01--  https://raw.githubusercontent.com/ytakahashi-statgen/expression_modifier_score_v2/refs/heads/main/ems_scaling_scores.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1051223 (1.0M) [text/plain]
Saving to: ‘ems_scaling_scores.csv’

ems_scaling_scores. 

,EMS_raw_score,Whole_Blood_EMSv2,Muscle_Skeletal_EMSv2,Liver_EMSv2,Brain_Cerebellum_EMSv2,Prostate_EMSv2,Spleen_EMSv2,Skin_Sun_Exposed_Lower_leg_EMSv2,Artery_Coronary_EMSv2,Esophagus_Muscularis_EMSv2,...,Brain_Spinal_cord_cervical_c-1_EMSv2,Brain_Substantia_nigra_EMSv2,Cells_Cultured_fibroblasts_EMSv2,Cells_EBV-transformed_lymphocytes_EMSv2,Kidney_Cortex_EMSv2,Minor_Salivary_Gland_EMSv2,Ovary_EMSv2,Small_Intestine_Terminal_Ileum_EMSv2,Uterus_EMSv2,Vagina_EMSv2
0,0.000,2.556027e-07,1.162706e-07,3.487263e-08,1.668640e-07,1.025745e-07,8.954207e-08,1.312655e-07,5.288330e-08,1.331129e-07,...,3.955548e-08,2.706677e-08,1.433407e-07,4.508196e-08,0.0,1.154782e-07,1.863451e-08,1.474391e-07,4.961639e-09,4.723821e-08
1,0.001,2.597645e-07,1.112291e-07,3.410651e-08,1.656341e-07,1.018860e-07,9.139348e-08,1.356897e-07,5.238486e-08,1.322462e-07,...,3.873323e-08,2.600449e-08,1.420262e-07,4.325035e-08,0.0,1.119999e-07,1.830622e-08,1.502013e-07,4.822321e-09,4.608864e-08
2,0.002,2.664786e-07,1.082820e-07,3.343280e-08,1.649444e-07,9.911833e-08,9.054958e-08,1.393615e-07,5.209269e-08,1.360159e-07,...,3.802814e-08,3.306689e-08,1.395010e-07,4.505086e-08,0.0,1.090032e-07,1.802021e-08,1.463158e-07,4.702461e-09,4.735540e-08
3,0.003,2.659103e-07,1.055412e-07,3.282376e-08,1.665127e-07,9.653016e-08,9.131524e-08,1.421061e-07,5.346654e-08,1.372970e-07,...,3.741678e-08,3.183584e-08,1.388541e-07,4.343469e-08,0.0,1.010502e-07,1.776513e-08,1.451037e-07,4.598581e-09,4.846618e-08
4,0.004,2.612606e-07,1.013437e-07,3.443550e-08,1.681012e-07,9.019851e-08,9.364118e-08,1.413469e-07,5.490442e-08,1.416084e-07,...,3.687059e-08,3.068532e-08,1.367327e-07,4.505016e-08,0.0,9.883611e-08,1.754499e-08,1.441393e-07,4.501694e-09,4.771880e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,0.946,6.203206e-04,1.023816e-03,3.404440e-04,9.473456e-04,3.618193e-04,4.034268e-04,1.608929e-03,2.395134e-04,9.802826e-04,...,2.636820e-04,2.190630e-04,8.740784e-04,1.214582e-04,0.0,1.008104e-04,2.826612e-04,2.147195e-04,1.428170e-04,1.152042e-04
947,0.947,6.492004e-04,1.072760e-03,3.525540e-04,9.786403e-04,3.802350e-04,4.559099e-04,1.718465e-03,2.563426e-04,1.059873e-03,...,2.650370e-04,2.178908e-04,9.271816e-04,1.221262e-04,0.0,1.057784e-04,2.961843e-04,2.179266e-04,1.478784e-04,1.153829e-04
948,0.948,7.465729e-04,1.206495e-03,3.774965e-04,1.002989e-03,3.787895e-04,4.894102e-04,1.871915e-03,2.792082e-04,1.167385e-03,...,2.652379e-04,2.234679e-04,1.037698e-03,1.369627e-04,0.0,1.038644e-04,3.214386e-04,2.271043e-04,1.575682e-04,1.188993e-04
949,0.949,9.142184e-04,1.356952e-03,3.595121e-04,9.802139e-04,4.051968e-04,5.304951e-04,1.981703e-03,2.917439e-04,1.274816e-03,...,2.720298e-04,2.174416e-04,1.088183e-03,1.558802e-04,0.0,1.044820e-04,3.336575e-04,2.292792e-04,1.675573e-04,1.225575e-04


In [ ]:
# raw score → scaling score
tmp2 = pd.DataFrame()
list_raw = df_ems_scale["EMS_raw_score"].values
for tissue in tqdm(tissues):
    list_scale= []
    for i in y_prob[tissue].values:
        idx = np.abs(np.asarray(list_raw) - i).argmin()
        list_scale.append(idx)
    tmp1 = pd.DataFrame(df_ems_scale.loc[list_scale,tissue+"_EMSv2"])
    tmp1.reset_index(drop=True,inplace=True)
    tmp2 = pd.concat([tmp2,tmp1],axis=1)
df_ems = pd.concat([y_prob[["id","ensg_id"]],tmp2],axis=1)
df_ems

  0%|          | 0/49 [00:00<?, ?it/s]

,id,ensg_id,Whole_Blood_EMSv2,Muscle_Skeletal_EMSv2,Liver_EMSv2,Brain_Cerebellum_EMSv2,Prostate_EMSv2,Spleen_EMSv2,Skin_Sun_Exposed_Lower_leg_EMSv2,Artery_Coronary_EMSv2,...,Brain_Spinal_cord_cervical_c-1_EMSv2,Brain_Substantia_nigra_EMSv2,Cells_Cultured_fibroblasts_EMSv2,Cells_EBV-transformed_lymphocytes_EMSv2,Kidney_Cortex_EMSv2,Minor_Salivary_Gland_EMSv2,Ovary_EMSv2,Small_Intestine_Terminal_Ileum_EMSv2,Uterus_EMSv2,Vagina_EMSv2
0,chr1_69134_A_G,ENSG00000223972,0.000091,0.000111,4.929314e-05,0.000099,8.887866e-05,8.421587e-05,0.000195,6.840032e-05,...,4.778142e-05,3.172801e-05,0.000120,2.095632e-05,9.757544e-06,4.178478e-05,0.000053,5.076613e-05,4.422975e-05,3.813196e-05
1,chr1_69134_A_G,ENSG00000227232,0.000095,0.000119,5.128956e-05,0.000103,9.324198e-05,8.828572e-05,0.000208,7.098468e-05,...,5.373486e-05,3.319710e-05,0.000129,2.137005e-05,1.045588e-05,4.272971e-05,0.000054,5.384837e-05,4.825628e-05,4.096300e-05
2,chr1_69134_A_G,ENSG00000278267,0.000092,0.000114,4.958279e-05,0.000100,8.979913e-05,8.594493e-05,0.000200,6.949866e-05,...,4.961143e-05,3.200011e-05,0.000123,2.129791e-05,9.880594e-06,4.212206e-05,0.000053,5.198456e-05,4.551924e-05,3.923995e-05
3,chr1_69134_A_G,ENSG00000243485,0.000095,0.000119,5.128956e-05,0.000103,9.324198e-05,8.828572e-05,0.000208,7.098468e-05,...,5.373486e-05,3.319710e-05,0.000129,2.137005e-05,1.045588e-05,4.272971e-05,0.000054,5.384837e-05,4.825628e-05,4.096300e-05
4,chr1_69134_A_G,ENSG00000237613,0.000097,0.000120,5.201230e-05,0.000103,9.500139e-05,9.003615e-05,0.000212,7.175469e-05,...,5.373486e-05,3.401712e-05,0.000129,2.137005e-05,1.083767e-05,4.421048e-05,0.000055,5.396625e-05,5.077606e-05,4.096300e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,chr1_69682_G_A,ENSG00000231702,0.000004,0.000002,6.486174e-07,0.000002,1.180593e-06,7.504672e-07,0.000002,5.794885e-07,...,2.579505e-07,3.298781e-07,0.000002,4.902361e-07,1.876188e-07,4.134155e-07,0.000001,8.741612e-07,7.048887e-07,4.581881e-07
346,chr1_69682_G_A,ENSG00000224969,0.000004,0.000002,6.495600e-07,0.000002,1.196039e-06,7.798903e-07,0.000002,5.776646e-07,...,2.572326e-07,3.593849e-07,0.000003,4.910285e-07,1.876188e-07,3.972145e-07,0.000001,8.737134e-07,7.348824e-07,4.738462e-07
347,chr1_69682_G_A,ENSG00000188157,0.000003,0.000002,6.313131e-07,0.000002,1.140502e-06,7.606325e-07,0.000002,5.889219e-07,...,2.227367e-07,3.580214e-07,0.000003,4.549552e-07,1.758987e-07,3.813285e-07,0.000001,8.555324e-07,7.503719e-07,4.505295e-07
348,chr1_69682_G_A,ENSG00000217801,0.000003,0.000002,6.564234e-07,0.000002,1.002550e-06,8.428063e-07,0.000002,4.768429e-07,...,2.174036e-07,3.530154e-07,0.000002,4.012761e-07,1.592855e-07,3.445274e-07,0.000001,8.614849e-07,6.602753e-07,4.901194e-07
